In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
!pip install xgboost

  Using cached xgboost-2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
Using cached xgboost-2.1.3-py3-none-manylinux2014_x86_64.whl (4.5 MB)


In [ ]:
# Import necessary libraries
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import accuracy_score

# Load dataset (example: breast cancer dataset)
X, y = load_svmlight_file('/mnt/homeGPU/joman/TFM-Paralelizacion/data/raw/epsilon_normalized')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert datasets to DMatrix (optimized format for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set parameters for GPU training
params = {
    'objective': 'binary:logistic',   # Binary classification
    'max_depth': 6,                    # Max depth of tree
    'learning_rate': 0.1,              # Step size
    'eval_metric': 'logloss',          # Evaluation metric
    'tree_method': 'gpu_hist',         # Use GPU-accelerated histogram-based algorithm
    'device': 'cuda'                  # Enable CUDA for GPU acceleration
          # Use GPU for prediction as well
}


# Training the model with GPU acceleration
num_round = 100  # Number of boosting rounds
bst = xgb.train(params, dtrain, num_round)

# Making predictions on the test set
y_pred = bst.predict(dtest)
y_pred_binary = (y_pred > 0.5).astype(int)  # Convert to binary classification

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f"Accuracy: {accuracy:.4f}")

/mnt/homeGPU/joman/.venv/lib/python3.10/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [1]:
import xgboost as xgb
print(xgb.__version__)


1.7.5
